# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/output_data/weather_data.csv")
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,dakar,0,SN,1580508714,31,14.69,-17.44,78.80,14.99
1,1,hithadhoo,59,MV,1580508841,72,-0.60,73.08,81.63,10.92
2,2,port alfred,95,ZA,1580508841,90,-33.59,26.89,72.00,7.00
3,3,tamuin,92,MX,1580508841,67,21.98,-98.75,70.93,3.40
4,5,carnarvon,0,AU,1580508841,72,-24.87,113.63,66.20,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_df[['Lat', 'Lng']]
weight = vacation_df['Humidity'].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temp = vacation_df.loc[(vacation_df['Max Temp'] <80) & (vacation_df['Max Temp'] >70)]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed'] <10]
ideal_cloud = ideal_wind.loc[ideal_wind['Cloudiness']==0] 
ideal_cloud

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
60,65,gao,0,ML,1580508931,12,16.64,1.64,74.43,9.42
220,246,sao filipe,0,CV,1580509304,45,14.90,-24.50,78.91,9.31
296,333,edd,0,ER,1580509440,58,13.93,41.69,79.09,4.36
303,342,tessalit,0,ML,1580509501,14,20.20,1.01,70.63,7.87
473,530,araouane,0,ML,1580509729,13,18.90,-3.53,72.43,9.84


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(ideal_cloud, columns=['City', 'Cloudiness', 'Country', 'Date',
                                             'Humidity', 'Lat', 'Lng', 'Max Temp',
                                             'Wind Speed'])
hotel_df['Hotel Name']=''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
60,gao,0,ML,1580508931,12,16.64,1.64,74.43,9.42,
220,sao filipe,0,CV,1580509304,45,14.90,-24.50,78.91,9.31,
296,edd,0,ER,1580509440,58,13.93,41.69,79.09,4.36,
303,tessalit,0,ML,1580509501,14,20.20,1.01,70.63,7.87,
473,araouane,0,ML,1580509729,13,18.90,-3.53,72.43,9.84,


In [7]:
params = {"radius": 5000,
          "types": "restaurant",
          "key": g_key
         }

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    
    params['location'] = f"{lat},{lon}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    names = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = names["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing result... skipping.")

Missing result... skipping.
Missing result... skipping.
Missing result... skipping.
Missing result... skipping.


In [14]:
#dropping areas where the hotel name column is missing

clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
220,sao filipe,0,CV,1580509304,45,14.9,-24.5,78.91,9.31,Tortuga


In [9]:
clean_hotel_df.to_csv('output_data/Hotels.csv', index=False)

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
#creating marker locations
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))